In [106]:
import requests
import json
import sys

# Change apikey with the apikey retrieved from deploit's webpage
apikey = "enter-your-api-key-here"
deploit_url = "https://deploit.lifebit.ai"

In [107]:
# This function will fetch the project id for a given project name
def fetch_deploit_project_id(apikey, project_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/projects".format(deploit_url), params=data)
    for project in json.loads(r.content):
        if project["name"] == project_name:
            return project["_id"]
    

In [108]:
project_id = fetch_deploit_project_id(apikey, "Demo")
if not project_id:
    sys.exit("Project id couldn't be find. Check that you provided a valid project name")

In [109]:
# This function will fetch the pipeline id for a given pipeline name
def fetch_deploit_pipeline_id(apikey, pipeline_name):
    data = {"apikey": apikey}
    r = requests.get("{}/api/v1/workflows".format(deploit_url), params=data)
    for pipeline in json.loads(r.content):
        if pipeline["name"] == pipeline_name:
            return pipeline["_id"]

In [110]:
pipeline_id = fetch_deploit_pipeline_id(apikey, "GWAS")
if not pipeline_id:
    sys.exit("Pipeline id couldn't be find. Check that you provided a valid pipeline name")

In [111]:
def send_job_request_to_deploit(apikey, pipeline_id, project_id):
    # Prepare api request for deploit to run a job
    headers = {
        "Content-type": "application/json",
        "apikey": apikey
    }

    pipeline_params = [
       {
         "name": "input_param",
         "prefix": "--",
         "parameterKind": "textValue",
         "textValue": "some value"
       }
    ]

    params = {
        "executionPlatform": "aws",
        "instanceType": "m2.2xlarge",
        "parameters": pipeline_params,
        "project": project_id,
        "workflow": pipeline_id
    }

    r = requests.post("{}/api/v1/jobs".format(deploit_url), data=json.dumps(params), headers=headers)
    return json.loads(r.content)["_id"]

In [112]:
job_id = send_job_request_to_deploit(apikey, pipeline_id, project_id)
if job_id:
    print("Job successfully sent to deploit. You can check the status " +
          "of the job in https://deploit.lifebit.ai/app/jobs/{}".format(job_id))

Job successfully sent to deploit. You can check the status of the job in https://deploit.lifebit.ai/app/jobs/generated_job_id
